In [1]:
#==================================
#   Run this cell at first
#==================================

%run ODE_DFLS.py

from sklearn import datasets
boston = datasets.load_boston()
X = boston.data
y = boston.target
y=y-np.mean(y)  # centered data

from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X)
X=standardScaler.transform(X)

from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

print(X.shape, X_test.shape)

(404, 13) (102, 13)


In [2]:
lam=.2
nn=200
gl = GroupLasso(
        groups=[1,1,1,2,2,2,3,3,4,4,5,5,5],
        group_reg=lam,
        l1_reg=0,
        scale_reg="group_size",
        supress_warning=True,
        n_iter=2000,
        fit_intercept=False,
        tol=1e-8)
sub=13
N=5

X_=X[:,:sub].copy()
y_=y.copy()
zcomp=[]
for i in np.arange(1,N+1):
    zcomp.append(sub-i)
X_[:,zcomp]*=0.
gl.fit(X_, y_)
w0=gl.coef_.flatten()
gl.fit(X[:,:sub], y)
w2=gl.coef_.flatten()

group1=[[0,0,1,2],[0,3,4,5],[0,6,7]]
group2=[[0,8,9],[0,10,11,12]]
for ele in group1:
    if w0[ele[1]]!=0:
        ele[0]=1
w1=w0.copy()

w1,tp=increFeature_GL(X[:,:sub-N],y,X[:,sub-N:],w0,group1,group2,lam,nn)

print("ODE_rmse=",np.sqrt(mean_squared_error(y_test,X_test@w1)))
print("batch_rmse=",np.sqrt(mean_squared_error(y_test,X_test@w2)))

print("ODE solution is",w1)
print("batch solution is",w2)

ODE_rmse= 5.3800527855169165
batch_rmse= 5.380202381581945
ODE solution is [ 0.          0.          0.          0.8362976  -0.76118379  2.70907174
 -0.04939485 -0.43047408  0.          0.         -1.84100084  0.83429077
 -3.06429927]
batch solution is [-0.          0.         -0.          0.83469808 -0.75240343  2.69807269
 -0.04881113 -0.44422872  0.         -0.         -1.84110063  0.83439683
 -3.07392447]
